In [1]:
from __future__ import division, unicode_literals

In [2]:
import os 
from pprint import pprint

The aim of the streamm package is to design a project with numerous calculations, run them on local and remote resources and collect the output for analysis, thus facilitating high-throughput computational material design.

To accomplish this, the directory structure is contained within a resource as a dictionary. Resources, structures, and forcefields are contained within a calculation object. Sets of calculations are contained within a project

So let's first create a resource object that we will use to set the directory locations of all the subsequent calculation objects 

In [3]:
import streamm

In [4]:
import streamm.calculations.resource as resource  

In [5]:
res_i = resource.Resource('local')

This sets the current working directory as the root/home directory by default

In [6]:
pprint(res_i.dir)

{u'home': '/Users/rlarsen/Development/streamm-tools/examples',
 u'launch': u'/Users/rlarsen/Development/streamm-tools/examples/scratch',
 u'materials': u'/Users/rlarsen/Development/streamm-tools/examples/materials',
 u'scratch': u'/Users/rlarsen/Development/streamm-tools/examples/scratch',
 u'scripts': u'/Users/rlarsen/Development/streamm-tools/examples/scripts',
 u'storage': u'/Users/rlarsen/Development/streamm-tools/examples/storage',
 u'templates': u'/Users/rlarsen/Development/streamm-tools/examples/templates'}


Let's create a new home directory called 'example_proj'

In [7]:
EXAMPLE_DIR = res_i.dir['home']

In [8]:
new_root = os.path.join(EXAMPLE_DIR,'example_proj')

In [9]:
res_i.set_home(new_root)

In [10]:
pprint(res_i.dir)

{u'home': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj',
 u'launch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch',
 u'materials': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/materials',
 u'scratch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch',
 u'scripts': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scripts',
 u'storage': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/storage',
 u'templates': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/templates'}


However, we want to use structures from our previous structures and forcefields examples, so let's set the materials directory to examples/

In [11]:
res_i.dir['materials'] = EXAMPLE_DIR

To write out input files we will use the templates provided in the streamm package

Set the template dir dictionary entry to the location of templates directory 

In [12]:
res_i.dir['templates'] =  os.path.join(EXAMPLE_DIR,'..','templates','')

In [13]:
print res_i.dir['templates']

/Users/rlarsen/Development/streamm-tools/examples/../templates/


This also contains the properties dictionary, which can be used to write .pbs scripts on clusters

In [14]:
pprint(res_i.properties)

{u'allocation': u'',
 u'exe_command': u'./',
 u'feature': u'24core',
 u'nodes': 1,
 u'nproc': 1,
 u'pmem': 1500,
 u'ppn': 1,
 u'queue': u'batch',
 u'walltime': 24}


By default the resource type is 'local'; however, setting type to 'ssh' will invoke an scp command when copying files

Okay create the directories we need for our calculation

In [15]:
res_i.make_dir()

Now we should have a directory 'example_proj/' with materials, scratch,   scripts,   storage and   templates directories 

We can create a gaussian calculation

In [16]:
import streamm.calculations.gaussian as gaussian  

In [17]:
calc_i = gaussian.Gaussian('methane_HF')

Set the resource and all the directories 

In [18]:
calc_i.set_resource(res_i)

In [19]:
pprint(calc_i.dir)

{u'home': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj',
 u'launch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_HF/',
 u'materials': '/Users/rlarsen/Development/streamm-tools/examples',
 u'scratch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_HF/',
 u'scripts': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scripts',
 u'storage': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/storage/methane_HF/',
 u'templates': u'/Users/rlarsen/Development/streamm-tools/examples/../templates/'}


Make the calculation directories

In [20]:
calc_i.make_dir()

Let's assign a structure to this calculation 

First copy the .xyz file from the materials directory to our scratch directory using the cp_file() function.

This takes an type and key to set the calc_i.files[type][key] dictionary 

In [21]:
file_type = 'input'
file_key = 'xyz'
file_name = "methane.xyz"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Generally, the materials directory is thought to contain a repository of material files, and local versions in the scratch directory should be made in case modifications are necessary 

Change to the scratch directory 

In [22]:
pprint(calc_i.dir['scratch'])

u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_HF/'


In [23]:
os.chdir(calc_i.dir['scratch'])

Read in methane .xyz file from the structures example

In [24]:
calc_i.strucC.read_xyz('methane.xyz')

In [25]:
print calc_i.strucC.n_particles

5


Now that we have a structure and parameters for each interaction we can create an input file for a simulation

Get the bash run script for Gaussian. By setting the file_key to run, this will be the script that executed when the run() function is called

In [26]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Get the .com template

In [27]:

file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)


Make sure we are in the scratch directory

In [28]:
pprint(os.getcwd())

'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_HF'


Load the template files into memory 

In [29]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

Set the properties strings in the template files to have the values we want

In [40]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

Replace the strings in the files['input']['com']

In [41]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

Add the name of the .com file to the properties, and replace the strings in the files['input']['run']

In [42]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.sh'%(calc_i.tag))

Save a .json file in the home directory 

In [43]:
os.chdir(calc_i.dir['home'])
calc_i.dump_json()

Go to scratch directory and see if there is a completed output file for the calculation

In [44]:
os.chdir(calc_i.dir['scratch'])
calc_i.check()

Check the status 

In [45]:
pprint("Calculation:{} has status:{}".format(calc_i.tag,calc_i.meta['status']))

u'Calculation:methane_HF has status:written'


If you have gaussian installed on your machine and g09 in your PATH you can run the bash script 

In [54]:
calc_i.run()

You can read in the data from the log file 

In [55]:
calc_i.add_file('output','log','{}.log'.format(calc_i.strucC.tag))

In [56]:
calc_i.check()
if(calc_i.meta['status'] == 'finished' ):
    calc_i.analysis()

Then compress the results and copy them to storage 

In [57]:
calc_i.store()

Next we can follow a similar procedure to run a LAMMPS MD simulation

In [46]:
import streamm.calculations.lammps as lammps  

In [47]:
calc_j = lammps.LAMMPS('methane_lmp')

Set the resource 

In [48]:
calc_j.set_resource(res_i)

Make directories

In [49]:
calc_j.make_dir()

In [50]:
pprint(calc_j.dir)

{u'home': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj',
 u'launch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_lmp/',
 u'materials': '/Users/rlarsen/Development/streamm-tools/examples',
 u'scratch': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scratch/methane_lmp/',
 u'scripts': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/scripts',
 u'storage': u'/Users/rlarsen/Development/streamm-tools/examples/example_proj/storage/methane_lmp/',
 u'templates': u'/Users/rlarsen/Development/streamm-tools/examples/../templates/'}


This takes an type and key to set the calc_i.files[type][key] dictionary 

In [51]:
file_type = 'input'
file_key = 'xyz'
file_name = "methane.xyz"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [52]:
os.chdir(calc_j.dir['scratch'])

Read in methane .xyz file from the structures example

In [53]:
calc_j.strucC.read_xyz('methane.xyz')

In [54]:
print calc_j.strucC.n_particles

5


Set the forcefield particletypes 

In [55]:
for pkey,p in calc_j.strucC.particles.iteritems():
    if( p.symbol == 'C' ):
        p.paramkey = 'CT'
    elif( p.symbol == 'H' ):
        p.paramkey = 'HC'

Set neighbor list 

In [56]:
calc_j.strucC.bonded_nblist = calc_j.strucC.guess_nblist(0,radii_buffer=1.25)

Find bonds and bond angles based on neighbor list 

In [57]:
calc_j.strucC.bonded_bonds()
calc_j.strucC.bonded_angles()

Copy the pickled forcefield parameter file to scratch and read it in

In [58]:
file_type = 'input'
file_key = 'param'
file_name = "oplsaa.pkl"
from_dirkey = 'materials'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [59]:
import streamm.forcefields.parameters as parameters 

In [60]:
calc_j.paramC = parameters.read_pickle('oplsaa')

In [61]:
print calc_j.paramC


    Parameters 
      LJ parameters 2 
      Bond parameters 2 
      Angle parameters 2 
      Dihedral parameters 1 
      Imporper Dihedral parameters 0 



In [62]:
for ptkey,pt in calc_j.paramC.particletypes.iteritems():
    print ptkey,pt,pt.unit_conf['energy'],pt.unit_conf['length']

0  CT epsilon:0.066 sigma:3.5 kCalmol ang
1  HC epsilon:0.03 sigma:2.5 kCalmol ang


In [63]:
for btkey,bt in calc_j.paramC.bondtypes.iteritems():
    print btkey,bt,bt.unit_conf['harm_bond_coeff'],pt.unit_conf['length']

0  bond  CT - HC type harmonic 
  harmonic r_0 = 1.080000 K = 367.000000 lammps index 0  gromacs index 0   kCalmolsqang ang
1  bond  CT - CT type harmonic 
  harmonic r_0 = 1.530000 K = 268.000000 lammps index 0  gromacs index 0   kCalmolsqang ang


In [64]:
for atkey,at in calc_j.paramC.angletypes.iteritems():
    print atkey,at,at.unit_conf['energy'],at.unit_conf['length']

0  angle  HC - CT - HC type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0   kCalmol ang
1  angle  HC - CT - CT type harmonic 
  harmonic theta_0 = 110.700000 K = 37.500000 lammps index 0  gromacs index 0   kCalmol ang


Use the set_ffparam() function to iterate through the structure container and set parameters based on ffkeys

In [65]:
calc_j.set_ffparam()

Now we have a structure that has forcefield parameters for each particle, bond and bond angle

Let's get the input file template 

In [66]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Bash run file 

In [67]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
calc_j.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

Got to scratch dir

In [68]:
os.chdir(calc_j.dir['scratch'])

Read in template files

In [69]:
calc_j.load_str('templates','in')
calc_j.load_str('templates','run')

Write LAMMPS data file

In [70]:
calc_j.write_data()

Replace properties strings in template and write template 

In [71]:
calc_j.replacewrite_prop('in','input','in','%s.in'%(calc_j.tag))

Set .in file in properties and write run script 

In [72]:
calc_j.properties['input_in'] = calc_j.files['input']['in']
calc_j.replacewrite_prop('run','scripts','run','%s.sh'%(calc_j.tag))

Save a .json file in the home directory 

In [73]:
os.chdir(calc_j.dir['home'])
calc_j.dump_json()

Go to scratch directory and see if there is a completed output file for the calculation

In [86]:
os.chdir(calc_j.dir['scratch'])
calc_j.check()

In [87]:
pprint("Calculation:{} has status:{}".format(calc_j.tag,calc_j.meta['status']))

u'Calculation:methane_lmp has status:written'


So now we have two calculations, let's put them in a project so we can operate on them both at the same time

In [88]:
import streamm.calculations.project as project  

In [89]:
import copy

In [90]:
proj_i = streamm.Project('example_proj')

In [91]:
proj_i.calculations[calc_i.tag] = copy.deepcopy(calc_i)
proj_i.calculations[calc_j.tag] = copy.deepcopy(calc_j)

Now we can check the status of each calculation with a single command 

In [92]:
proj_i.check()

Calculation methane_lmp has status written
Calculation methane_HF has status written


We can run each simulation

In [93]:
proj_i.run()

/Users/tkemper/Development/STREAMM/streamm-tools/examples/example_proj/scratch/methane_lmp
/Users/tkemper/Development/STREAMM/streamm-tools/examples/example_proj/scratch/methane_HF


We can tar up the results and copy the tar files to a storage location

In [94]:
proj_i.store()

Neat-O!